## Selenium을 이용한 크롤링
#### 멜론티켓

In [1]:
from selenium import webdriver
import time

driver = webdriver.Chrome('c:/chromedriver.exe')

In [5]:
def get_data():
    
    p_info={'Type':'', 'Title':'', 'TicketOpen':'', 'Venue':'', 'StartDate':'', 'EndDate':'', 'Time':'',
           'RunningTime':'', 'Price':'', 'BuyTicket':'멜론', 'PosterSrc':'', 'RegisteredTime':'', 'Performer':''}
#     p_info={'Type':'', 'Title':'', 'Subtitle':'', 'Venue':'', 'StartDate':'', 'EndDate':'', 'Performer':''}
    performer=[]
    
    # Type
    p_info['Type']=driver.find_element_by_xpath('//*[@id="conts"]/div/div[1]/div[1]/div[2]/div[2]/dl[1]/dd[3]').text
    
    # Title
    p_info['Title']=driver.find_element_by_xpath('//*[@id="conts"]/div/div[1]/div[1]/div[2]/p[2]').text
    
#     # Subtitle
#     p_info['Subtitle']=driver.find_element_by_xpath('//*[@id="conts"]/div/div[1]/div[1]/div[2]/p[2]').text

    # RunningTime
    p_info['RunningTime']=driver.find_element_by_xpath('//*[@id="conts"]/div/div[1]/div[1]/div[2]/div[2]/dl[1]/dd[2]').text
    
    # PosterSrc
    p_info['PosterSrc']=driver.find_element_by_xpath('//*[@id="conts"]/div/div[1]/div[1]/div[1]/img').get_attribute('src')
    
    # Time
    p_info['Time']=driver.find_element_by_xpath('//*[@id="conts"]/div/div[3]/div[1]/div[2]/p[2]/span').text.split(') ', 2)[1]
        
    # Price
    p_info['Price']=driver.find_element_by_xpath('//*[@id="conts"]/div/div[3]/div[1]/div[3]/ul/li').text
    p_info['Price'].replace('\n', ' ')
    
    # Venue
    try:
        p_info['Venue']=driver.find_element_by_xpath('//*[@id="performanceHallBtn"]/span[1]').text
    except:
        p_info['Venue']=' '
    
    # Date
    temp_date=driver.find_element_by_xpath('//*[@id="periodInfo"]').text
    if '-' in temp_date:
        p_info['StartDate']=temp_date.split(' ', 2)[0]
        p_info['EndDate']=temp_date.split(' ', 2)[2]

    # Performer
    li_artist=driver.find_elements_by_class_name('list_artist')
    for i in li_artist:
        if i.find_element_by_tag_name('li') != 'no_artist':
            performer.append(i.find_element_by_class_name('singer').text)
    p_info['Performer']=performer
    
        
    print(p_info)
    p_data.append(p_info)

In [6]:
def get_link():
    
    temp_li=driver.find_element_by_xpath('//*[@id="perf_poster"]')
    
    try:
        for p_block in temp_li.find_elements_by_tag_name('li'):
            p_link.append(p_block.find_element_by_tag_name('a').get_attribute('href'))
    except:
        print('error')
            
        

In [7]:
plays=[]       # 전체 공연정보 리스트
plays_type=['GENRE_CON', 'GENRE_ART', 'GENRE_CLA', 'GENRE_EXH']

for i in range(0,len(plays_type)):
    
    # 장르에 따른 '전체보기 - 상품명순' 페이지 접근
    driver.get("https://ticket.melon.com/concert/index.htm?genreType="+plays_type[i])
    time.sleep(1)
    
    p_link=[]       # 장르별 공연 상세페이지 링크 리스트
    p_data=[]       # 장르별 공연정보 리스트
    
    # 최신순으로 정렬
    driver.find_element_by_xpath('//*[@id="sort_lately_admin"]/a').click()
    
    # 각 공연의 상세페이지 링크 p_link에 가져오기
    get_link()
    
    for j in p_link:
        
        # 상세페이지 링크에 접근
        driver.get(j)
        time.sleep(0.5)
            
        # 공연정보 데이터를 p_data에 가져오기
        get_data()
        
        # 전체 공연 리스트에 p_data 추가하기 (이중리스트)
        plays.append(p_data)


error
{'Type': '콘서트', 'Title': '소년，소녀를 만나다 VII', 'TicketOpen': '', 'Venue': '성수아트홀 ', 'StartDate': '2021.01.02', 'EndDate': '2021.01.02', 'Time': '오후 6시', 'RunningTime': '90분', 'Price': '전석\n55,000원', 'BuyTicket': '멜론', 'PosterSrc': 'https://cdnticket.melon.co.kr/resource/image/upload/product/2020/12/20201204130320fcc7ee7d-a451-4432-b008-3c980d1eb654.jpg/melon/resize/180x254/strip/true/quality/90/optimize', 'RegisteredTime': '', 'Performer': ['재주소년']}
{'Type': '콘서트', 'Title': '에이프릴 세컨드 단독 콘서트 〈1월의 돌고래：Do You Wanna〉', 'TicketOpen': '', 'Venue': '벨로주 홍대 ', 'StartDate': '2021.01.10', 'EndDate': '2021.01.10', 'Time': '오후 5시', 'RunningTime': '90분', 'Price': '전석\n49,000원', 'BuyTicket': '멜론', 'PosterSrc': 'https://cdnticket.melon.co.kr/resource/image/upload/product/2020/12/2020120411125080c54611-020c-48a2-af1d-1fa9679f3261.jpg/melon/resize/180x254/strip/true/quality/90/optimize', 'RegisteredTime': '', 'Performer': ['에이프릴 세컨드']}
{'Type': '콘서트', 'Title': '워킹애프터유 연말단독공연：FAREWELL 2020', 'TicketOp

IndexError: list index out of range